In [1]:
import os
from pyciemss.Ensemble.interfaces import (
    load_and_sample_petri_ensemble, load_and_calibrate_and_sample_ensemble_model
)

In [2]:
DEMO_PATH = "../../notebook/integration_demo/"
ASKENET_PATH_1 = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"
ASKENET_PATH_2 = "../../test/models/AMR_examples/SIDARTHE.amr.json"

ASKENET_PATHS = [ASKENET_PATH_1, ASKENET_PATH_2]

In [3]:
def solution_mapping1(model1_solution: dict) -> dict:
    return model1_solution


def solution_mapping2(model2_solution: dict) -> dict:
    mapped_solution = {}
    mapped_solution["S"] = (
        model2_solution["Susceptible"]
        + model2_solution["Recognized"]
        + model2_solution["Threatened"]
    )

    mapped_solution["I"] = (
        model2_solution["Infected"]
        + model2_solution["Ailing"]
        + model2_solution["Diagnosed"]
    )

    # Model 1 doesn't include dead people, and implicitly assumes that everyone who is infected will recover.
    mapped_solution["R"] = (
        model2_solution["Healed"] + model2_solution["Extinct"]
    )

    return mapped_solution


solution_mappings = [solution_mapping1, solution_mapping2]

## load_and_sample_ensemble_model

In [4]:
weights = [0.5, 0.5]
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run sampling
ensemble_samples = load_and_sample_petri_ensemble(
    ASKENET_PATHS, weights, solution_mappings, num_samples, timepoints
)

# Save results
ensemble_samples.to_csv(os.path.join(DEMO_PATH, "results_petri_ensemble/sample_results.csv"), index=False)

/Users/zuck016/Projects/Proposals/ASKEM/build/pyciemss/src/pyciemss/PetriNetODE/base.py:330: UserWarning: Parameter (('Infected', ('identity', 'ido:0000511')), ('Healed', ('identity', 'ido:0000592')), 'NaturalConversion', 'rate') has value None and will be set to Uniform(0, 1)
  warnings.warn(warnings_string)


## load_and_calibrate_and_sample_ensemble_model

In [5]:
data_path = os.path.join(DEMO_PATH, "data.csv")
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run the calibration and sampling
calibrated_samples = load_and_calibrate_and_sample_ensemble_model(
    ASKENET_PATHS,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=1000,
    num_iterations=350,
)

# Save results
calibrated_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri_ensemble/calibrated_sample_results.csv"), index=False
)

/Users/zuck016/Projects/Proposals/ASKEM/build/pyciemss/src/pyciemss/PetriNetODE/base.py:330: UserWarning: Parameter (('Infected', ('identity', 'ido:0000511')), ('Healed', ('identity', 'ido:0000592')), 'NaturalConversion', 'rate') has value None and will be set to Uniform(0, 1)
  warnings.warn(warnings_string)


iteration 0: loss = 66.16035544872284
iteration 25: loss = 42.31934851408005
iteration 50: loss = 33.17540234327316
iteration 75: loss = 27.15283751487732
iteration 100: loss = 21.151540756225586
iteration 125: loss = 17.88287889957428
iteration 150: loss = 19.490506768226624
iteration 175: loss = 16.146777868270874
iteration 200: loss = 16.498568773269653
iteration 225: loss = 15.53571480512619
iteration 250: loss = 15.07484495639801
iteration 275: loss = 14.595208406448364
iteration 300: loss = 14.831865012645721
iteration 325: loss = 15.439243197441101
